In [1]:
import os 
import shutil
import numpy as np
import glob
import cv2
import tensorflow as tf

from waymo_open_dataset import dataset_pb2 as open_dataset
from tqdm.notebook import tqdm

tf.enable_eager_execution()

In [2]:
FRONT = 0
def video_generation(folder, frames):
    imgs = []

    for frame in frames:
        image = frame.images[FRONT]
        img = tf.image.decode_jpeg(image.image)
        imgs.append(img.numpy())
    
    img = imgs[0]
    height, width, _ = img.shape
    size = (width, height)
    out_video = folder + 'video.mp4'
    fps = 10
    
    out = cv2.VideoWriter(out_video, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    
    for img in imgs:
        gbr = img[...,::-1].copy()
        out.write(gbr)
    out.release()



In [ ]:
train_path = '/home/dataset/training/*/*.tfrecord'
FRONT = 0
train_prefix = '/home/dataset/videos/training/'

train_tfs = glob.glob(train_path)

for file in tqdm(train_tfs):
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    
    tar_name = file.split('/')[4]
    tar_path = train_prefix + tar_name + '/'
    
    if not os.path.exists(tar_path):
        os.mkdir(tar_path)
        
    seg_name = file.split('/')[5][:-9]
    seg_path = tar_path + seg_name + '/'
    
    if os.path.exists(seg_path):
        shutil.rmtree(seg_path)
    os.mkdir(seg_path)
    
    frames = []
    for idx, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)
    
    video_generation(seg_path, frames)

In [ ]:
test_path = '/home/dataset/validation/*/*.tfrecord'
test_prefix = '/home/dataset/videos/validation/'

test_tfs = glob.glob(test_path)

for file in tqdm(test_tfs):
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    
    tar_name = file.split('/')[4]
    tar_path = test_prefix + tar_name + '/'
    
    if not os.path.exists(tar_path):
        os.mkdir(tar_path)
        
    seg_name = file.split('/')[5][:-9]
    seg_path = tar_path + seg_name + '/'
    
    if os.path.exists(seg_path):
        shutil.rmtree(seg_path)
    os.mkdir(seg_path)
    
    frames = []
    for idx, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        frames.append(frame)
    
    video_generation(seg_path, frames)
        